# Lecture 5: Sources of Risk

Previously we covered time series analysis and how it can be used to perform techniques like volatility forecasting and modeling stocks prices. We will now shift towards describing the sources of risk, which is heavily dependent on the notion of a probability space. 

The main topics of discussion are:

 1. Probability Spaces
 2. Fundamental Risk Factors
 3. Risk Mapping
 
**Disclaimer**: Unless serious modification will be made in the future, this lecture is a terrible read and you might as well read 2.1 of _QRM_ by McNeil.


In [29]:
#Importing standard libraries


import numpy as np
from matplotlib import pyplot as plt 
import scipy as sci
import pandas as pd

## Probability Spaces

Before discussing fundamental risk factors, it is important to first define a probability space, or a mathematical object that is formally defined by a three-tuple 

($\mathbf{\Omega}, \mathcal{F}, \mathbb{P}$), where

 1. $\mathbf{\Omega}$ is the sample space, or the set of all possible outcomes of a random experiment
 2. $\mathcal{F}$ is the event space also called $\sigma$-algebra, or a subset of the superset of $\mathbf{\Omega}$
 3. $\mathbb{P}: \mathcal{F} \rightarrow [0,1]$ is a probability function that assigns a number between 0 and 1 to any event. 
 

For example, consider a fair dice. Its sample space $\mathbf{\Omega}$ is $\{1,2,3,4,5,6\}$. One could define the $\sigma$-algebra to be the set of all subsets of $\mathbf{\Omega}$ which contains elements such as $\{5\}$ and $\{2,4,6\}$ (all possible even outcomes). The probability function $\mathbb{P}$ then maps such elements to their respective probability of occurence, eg. $\{5\} \rightarrow 1/6$ and $\{2,4,6\} \rightarrow 3/6 = 1/2$.

Such a construct, equipped with a set of axioms called the probability axioms build up to probability theory.

## Fundamental Risk Factors

A good starting point for describing risk sources is by looking at fundamental risk factors such as interest rates and market movements as well as how we can use them to build a risk model. We describe future enviromental conditions by a value as a function of a future time-point $V(s)$, which acts as a random variable in a probability space $(\mathbf{\Omega}, \mathcal{F}, \mathbb{P})$.


Given a time horizon $\Delta$ (e.g 1 day), the loss of an investment in the period $[s, s + \Delta]$ is given by the loss function 

$$L_{[s, s+ \Delta]} = -(V(s + \Delta) - V(s))$$

where $V(s + \Delta)$ is the future value and $V(s)$ is the present value. 


The loss $L_{[s, s + \Delta]}$ at time $s+\Delta$ is simply observable (i.e not stochastic), although at time $s$ it is not observable, meaning we need to treat it as an _unconditional_ loss distribution, which assumes we dont have all information up to time $s$. So we can describe what is commonly called the P/L Distribution as $V(s + \Delta) - V(s)$ which is essentally $rv - L_{[s,s+\Delta]}$ where $rv$ denotes random variable and $L_{[s,s+\Delta]}$ denotes a loss distribution. 


Although in Risk Management, one is usually only concerned with the losses (i.e an insurance business cannot profit from car thefts), so we concern ourselves with only negative values of asset change.

By assuming equidistant time periods, one can reduce $L_{[s,s + \Delta]}$ to $L_{t+1}$ where

$$L_{t+1} = -(V(t+1) - V(t))$$

For example when we look at the daily losses of a position or portfolio, we set $\Delta = 1/365$ (e.g FX) or $\Delta = 1/252$ (e.g equities) or various other Day count conventions that factor in things like holidays and leap years.


Note that values $V_t$ and $V_{t+1}$ denotes random variables that model the value of a portfolio at $t$ and $t+1$ respectively, whilst $L_{t+1}$ denotes the loss occured from $t$ to $t+1$. Given a scenario with $d$ risk factors taken into account, $V_t$ will be a random variable as a function of time and $\vec{Z_t}$, a $d$-dimensional random vector which encompasses all risk factors, i.e

$$V_t: \mathbb{R}^+ \times \mathbb{R}^d \rightarrow \mathbb{R}: = f(t, \vec{Z_t})$$


The form of $V_t(t, \vec{Z_t})$ be as simple as return $\times$ investment value, but could also get very complicated as one introduces instruments such as options. Common risk factors include interest rates, exchange rates, commodity prices and economic ratios. The mapping of such factors to a value function $V_t$ is called risk mapping, which will be covered in more detail in the next section.

#### Python Implementation

In [381]:
###Think of something -- at this point this might as well be a pdf

## Risk Mapping

As described above, given a set of risk factors $\vec{Z_t} = \begin{pmatrix}Z_{t1} \\ \vdots \\ Z_{td}\end{pmatrix}$ at time $t$, the goal of risk mapping is to define a function $f(t, \vec{Z_t})$ that outputs a random variable called the value, $V_t$. We can then represent a loss $L_{t+1}$ as

$$L_{t+1} = -(f(t+1, \vec{Z}_{t+1} + \vec{X}_{t+1}) - f(t, \vec{Z}_t))$$

where $(\vec{X}_t)_{t \in \mathbb{N}}$ is defined as the process of changes of risk factors, $\vec{X}_t  =\vec{Z}_{t+ 1} - \vec{Z}_t$. Since $\vec{Z}_t$ is known at time $t$, then the loss distribution is determined soley by the distribution of changes $X_{t+1}$. Given that, we can define a _loss operator_ $l_{[t]}: \mathbb{R}^d \rightarrow \mathbb{R}$ which takes the following form:

$$l_{[t]}(\vec{x}) = -(f(t+1, \vec{Z}_{t+1} + \vec{x}) - f(t, \vec{Z}_t))$$

where $\vec{x} \in \mathbb{R}^d$ is some $d$-dimensional vector. Using this definition, we define the loss function as $l_{[t]}(\vec{X}_{t+1})$

<hr>


Up until this point, we have made no assumptions or statements about the _mapping_ $f(t,\vec{Z}_{t})$. Although, under the assumption that it is differentiable, one can define a [linear approximation](https://web.ma.utexas.edu/users/m408n/CurrentWeb/LM3-10-2.php#:~:text=Derivatives%20can%20be%20used%20to,(a)x%E2%88%92a.&text=whenever%20x%20is%20close%20to%20a.,linearization%20of%20f(x) ) to the loss function $L^\Delta_{t+1}$ as

$$L^\Delta_{t+1} = -\left(\frac{\partial f}{\partial t} + \sum_{i=1}^d \frac{\partial f}{\partial z_i}X_{t+1, i}\right) $$


which, similarly to what has done above, can be turned into the _linearized loss operator_ defined by

$$l^\Delta_{[t]}(\vec{x}) = -\left(\frac{\partial f}{\partial t} + \sum_{i=1}^d \frac{\partial f}{\partial z_i}x_{ i}\right)$$

Given that these are based on linear approximations, one can easily show that for linear mappings $f^{''} = 0$, it is exact and for highly non-linear mappings, it is a bad approximation. 


### Example: Stock Portfolio

For the first example of performing a risk mapping, we will be looking at a simple portfolio containing $d$ stocks, with $\lambda_i$ denoting the number of units of stock $i$ at time $t$. A stock $i$ has a price that will be modelled by $(S_t)_{t \in \mathbb{N}}$ and, as per standard practice in QRM, the risk factor $Z_t$ will be set as the logarithm of the price $Z_t = \ln{S_{t,i}}$ and concequently $X_{t+1} = \ln{S_{t+1,i}} - \ln{S_{t,i}}$. In other words, the change in the risk factor of a given stock $i$ is its future log return.

The value of the portfolio is thus given by

$$V_t = \sum_{i=1}^d \lambda_i e^{Z_{t,i}}$$

which is the weighted sum of the prices $S_{t,i} = e^{\ln{S_{t,i}}} = e^{Z_{t,i}}$. This value function has a corresponding loss of 

$$L_{t+1} = -(V(t+1) - V(t)) = -\sum_{i=1}^d \lambda_i S_{t,i}(e^{X_{t+1,i}} - 1)$$


which, taken to a first order approximation, gives

$$L_{t+1}^\Delta = \lambda_iS_{t,i}X_{t+1,i} = -V_t\sum_{i=1}^d w_{i,t}X_{t+1,i} = -V_t(\vec{w}_t.\vec{X}_{t+1})$$

The corresponding linearized loss operator is thus

$$l^\Delta_{[t]}(\vec{x}) = -V_t(\vec{w}_{t} . \vec{x})$$


where $w_i$ denotes the weight of the $i$-th stock in the overall portfolio. 

Given a mean vector  and a covariance matrix, it is very easy to calculate the moments of the loss distribution $\mathbb{E}[L_{t+1}]$ and $var(L_{t+1})$. For example, supposing the random vector $\vec{X}$ has a corresponding mean vector $\vec{\mu}$ and covariance matrix $\mathbf{\Sigma}$, we can use the general rules for means and variances of linear combinations to arrive at

$$\mathbb{E}[l^\Delta_{[t]}(\vec{X})] = -V_t(\vec{w}.\vec{\mu})$$

and 

$$var\left(l^\Delta_{[t]}(\vec{X})\right) = V^2_t\vec{\mu}^T\mathbf{\Sigma}\vec{\mu}$$

(I apologize for my poor choice of vector notation, but I'm 5 lectures in)

Note: Assuming a normal distribution for $X_t$, then the losses must also be assumed to be normally distributed. 

### Example: Euro Call Option

Now let us consider a portfolio of Euro call options on a stock $S$ (that is non-dividend paying) with maturity date $T$ and exercise price $K$. We assume the valuation of the portfolio to be based on the Black-Scholes model, with price function $C^{BS}$ given by

$$C^{BS}(s,S|r,\sigma,K,T) = S\Phi(d_1) - Ke^{-r(T-s)}\Phi(d_2)$$

where $\Phi( .)$ is the standard normal distribution function (dt), r represents the continuously compounded risk-free interest rate, $\sigma$ denotes the annualized volatility of the underlying stock, and $d_1 = \frac{\ln{S/K} + (r + \frac{1}{2}\sigma^2)(T - s)}{\sigma\sqrt{T-s}}$ and $d_2 = d_1 - \sigma\sqrt{T - s}$. We will be interested in daily losses and, for reasons mentioned in section 2, will set $\Delta = 1/250$.

For obvious reasons, a good risk factor to choose for this model is the log price of the underlying. Although in this model, the interest rate $r$ and the volatility $\sigma$ are assumed to be constant, which in practice, is very rarely valid. For this reason, we define a risk factor vector 

$$\vec{Z}_t = \begin{pmatrix}\ln{S_t} \\ r_t \\ \sigma_t\end{pmatrix}$$

with corresponding risk-factor-change vector

$$\vec{X}_{t+1} = \begin{pmatrix}ln{S_{t+1}} - \ln{S_t} \\ r_{t+1} - r_t \\ \sigma_{t+1} - \sigma_t\end{pmatrix}$$


The value of the call option at day $t$ is given by

$$V_t = C^{BC}(t\Delta, S_T, r_t,\sigma_t, K, T)$$

which gives us a linearized loss of 

$$L^\Delta_{t+1} = -(C_s^{BS}\Delta + C_S^{BS}S_tX_{t+1,1} + C_r^{BS}S_tX_{t+1,2} + C_\sigma^{BS}S_tX_{t+1,3})$$

where $C^{BS}_i = \frac{\partial C^{BS}}{\partial i}$. These partial derivatives are very important quantities for traders and analysts that they are given names:

<img src="assets/greeks.png" width=500>

or in english:

 - Delta: sensitivity to stock price
 - Theta: sensitivity to time
 - Rho: sensitivity to interest rate
 - Vega: Sensitivity to volatility
 
Also note that in the figure above, $V$ means $C^{BS}$, and that Delta is usually taken to be the most important risk measure in option portfolios. Sometimes the loss function is highly non-linear, rendering its linearization to be a bad approximation. For this reason, other approximations such as the delta-gamma approximations are used. 

### Example: Bond Portfolio 

### Example: Credit Portfolio

### Python Implementation


Probably do some linearized loss calculator for options (or do some given )